# GRID SEARCH VE RANDOM FOREST REGRESSION ALGORİTMALARI İLE ARABA FİYATLARI TAHMİNİ

Bu yazıda regresyon algoritmalarından birisi olan ***Random Forest Regression*** algoritması ile araba
fiyatlarının tahmin edilmesini sağlayacağız.

Veri seti olarak kullanacağımız csv dosyasına [buradan](https://github.com/arifegulsah/Regression-Cars/blob/main/cars.csv) ulaşabilirsiniz.
<br>


# 1. VERİ ÖNİŞLEME

In [7]:
import pandas as pd
import csv

data = "cars.csv"
data =pd.read_csv(data)
data.head()

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438


Bu veri seti 6 farklı sütundan ve 964 farklı satırdan oluşmaktadır. İncelediğimiz zaman satışı yapılmış
arabaların yaşları, tipleri, kilometresi, ne kadara satıldığı gibi özelliklerinin bulunduğunu görmekteyiz.
Projemizin arabaların satış fiyatlarını tahmin etme üzerine olduğundan da anlaşılacağı üzere, satış fiyatı
yani “sales” başlıklı sütunun **classsımız** yani outputumuz, diğer sütunların ise **features** yani girdilerimiz
olduğunu anlayabilmekteyiz.
<br> <br>
Veri ön-işlememizi yapabilmek için yapmamız gereken adımlar:
1. Gerekli kütüphanelerimizin ve veri setimizin import edilmesi.
2. Veri setimizi sonrasında kullanılabilmesi için features ve class olarak ayrılması.
3. Veri setine, yüksek ve düşük sayıları aynı anda içerdiğinden ötürü feature-scaling uygulanması.
4. Veri setinin train ve test olarak ayrılması.

In [8]:
import numpy as np
import matplotlib.pyplot as plt

dataset = pd.read_csv('cars.csv')
X = dataset.iloc[:, 0:5].values
y = dataset.iloc[:, 5].values

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

## 2. ALGORİTMANIN KULLANIMI


Sıra Random Forest Regression (RFR) algoritmasını kodumuza dahil etme vaktinde.<br><br>
RFR algoritması, karar vermek için birden fazla Decision Trees’nin (Karar Ağacı) nitelik özelliklerini
kullanan "Tree" tabanlı bir algoritmadır. Karar Ağaçları, çok sayıda kayıt içeren bir veri kümesini, bir dizi
karar kuralları uygulayarak daha küçük kümelere bölmek için kullanılan bir yapıdır. Yani basit karar
verme adımları uygulanarak, büyük miktarlardaki kayıtları, çok küçük kayıt gruplarına bölerek
kullanılan bir yapıdır denilebilir. RFR ise, nihai çıktıyı oluşturmak için birden fazla Karar Ağacının çıktısını
birleştirmektedir.<br><br>
RFR, *scikit-learn* kütüphanesinde ensemble modülünün bir sınıfı olarak tanımlanmıştır. Sınıfımızın ismi
ise “RandomForestRegressor”dır. Bu sınıftan yaratacağımız ve **“regressor”** ismini vereceğimiz nesne,
bizim makine işimizi yapacak.<br><br>
Modelimizi eğitmek için öncelikle bu sınıftan regressor adında bir nesne oluşturuyoruz. Daha sonra bu
nesnenin **fit()** metoduna X, y değişkenlerimizi parametre olarak veriyoruz. Böylelikle makinemizi
kurmuş oluyoruz.

In [9]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 50, random_state = 1)
regressor.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, random_state=1)

Kodumuzda geçen “n_estimator” parametresi, makinemiz için kaç tane karar ağacı kullanılacağını
belirtmeye yaramaktadır. İstenilen herhangi bir değer tanınabilir. Varsayılan değeri 100’dür. Biz burada
deneme amaçlı 50 verdik.<br><br>
Modelimize vermiş olduğumuz X_train ve y_train değişkenleri ile regressor isimli makinemiz öğrenmiş
oldu. Şimdi öğrenimi test etmek amaçlı **predict()** fonksiyonu ile modelin daha önce karşılaşmadığı
**X_test** verilerini tahmin ediyoruz. Sonuçlarımızı karşılaştırabilmek için **y_pred** isimli değişkende
tutuyoruz. 

In [13]:
y_pred = regressor.predict(X_test)

Şimdi ise “Cross-Validation” tekniği kullanılarak modelimizin performansını daha objektif ve doğru bir
şekilde değerlendirelim. Bunun için kullanacağımız fonksiyon cross_val_score()’dur.<br><br>
Fonksiyonun dönütünü accuracies ismini verdiğimiz değişkende tutarak Cars veri setimizi 10 farklı
parçaya bölelim ve ortalama doğruluk değerlerini hesaplayalım.<br><br>
Bu hesaplamalar sonucunda böldüğümüz farklı parçaların performanslarını ayrı ayrı gözlemleyebiliriz


In [14]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_train , y = y_train, cv = 10)
accuracies.mean()
accuracies.std()

0.04166011462440495

In [18]:
accuracies

array([0.89279819, 0.86694113, 0.8570782 , 0.89906074, 0.90129845,
       0.90392387, 0.9305184 , 0.93024772, 0.78792764, 0.93059886])

## 3. GRID SEARCH ALGORİTMASI

Modelimizi geliştirmek için kullandığımız fonskiyonların parametlerinin seçilmesi, modeli tasarlayan
kişiye bırakılmıştır. Ne olması gerektiği, modeli tasarlayan kişiye bırakılmış, probleme ve veri setine
göre değişiklik gösteren bu parametreler, hiper-parametre olarak adlandırılmaktadır. Hiper
parametrelerden bazıları sonsuz sayıda değer alabilecek konumdadırlar. Bununla birlikte biz, problem
hakkında sahip olduğumuz ön bilgileri kullanarak hiper parametrelerin alabilecekleri değerler için
kendimiz aralıklar belirleyebiliriz.<br><br>
**Grid Search** isimli algoritma ile hiper parametre seçim işleminde; belirlenen aralıkta bulunan tüm
değerlerin kombinasyonları için makina eğitilip sonuçlar gözlenir. Duruma göre en iyi kombinasyon
hiper parametre grubu olarak seçilir.<br><br>
Şimdi de modelimizin hangi hiper parametreler ile daha efektif bir sonuç alacağını hızlıca
hesaplayabilmek için Grid Search algoritmasını kullanalım.<br><br>
Bu algoritmayı doğru bir şekilde kullanabilmek için öncesinde, makinemizi oluştururken kullandığımız
ana algoritmanın parametre özelliklerini iyi bilmek gerekmektedir. Bizim bu çalışmamızda kullandığımız
Random Forest Regression algortimasının alabileceği parametreleri ve değerlerini şu [linkten](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
inceleyebiliriz.<br><br>
Burdan da görüleceği üzere, n_estimators parametresi haricinde default olarak aldığımız diğer birçok
parametre mevcut. Bu parametrelerin her birini Grid Search ile kolaylıkla ve hızlı bir şekilde deneyelim.<br><br><br>
Algoritmamız sci-kit kütüphanesinde model_selection modülünün altında bulunmakta ve
GriedSearchCV ismindedir. Import ediyoruz.

In [19]:
from sklearn.model_selection import GridSearchCV

Daha sonrasında yapacağımız iş parametrelerimizi belirlemek olacaktır. Bunun için “n_estimators”,
“criterion”, ve “max_features” parametrelerimizi ele alalım. 

In [20]:
parameters = [{'n_estimators': [250], 'criterion': ['mse', 'mae'], 'max_features' : ['auto', 'sqrt', 'log2',
None]}]

Şeklinde yazarak aslında, regressorisimli makinamızda daha önce 50 olarak belirlediğimiz **n_estimators**
yani ağaç sayısını **1 ile 250** arasında denemek istediğimizi; **criterion** kısmına **mse** ve **mae** yazarak
ortalama kare hatası veya ortalama mutlak hata şeklinde 2 farklı metrik denemek istediğimizi;
**max_features** için ise alınabilecek olan **auto, sqrt, log2** ve **None** değerlerinin hepsini denemek
istediğimizi belirtmiş oluyoruz.<br><br>
Hiper paramterelerimizi istediğimiz şekilde belirledikten sonra sıra geldi bu parametreleri teker teker
modelimizde denemeye. Bunun için aşağıdaki kodu yazıyoruz.

In [21]:
grid_search = GridSearchCV(estimator = regressor,
 param_grid = parameters,
 scoring = 'r2',
 cv = 10,
 n_jobs = -1) 

GridSearchCV için girilen değerler:
1. estimator = modelimiz
2. param_grid = daha önce belirlediğimiz parametreler
3. cv = cross-validation stratejisini belirler ve default olarak 5 gelir
4. n_jobs = paralel olarak çalıştırılacak iş sayısını belirler, -1 ise tüm işlemcileri kullanmak <br>

anlamlarına gelmektedir. Bu ve bunun gibi farklı parametreler de mevcuttur. Yine daha detaylı bir
araştırma için aşağıdaki [linkten](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) yararlanabiliriz. <br><br>
Şimdi de grid_search ismini verdiğimiz veparametrelerini belirlediğimiz yeni modelimize X_train ve
y_train verilerimizi fit ediyoruz. Daha sonrasında en iyi doğruluk değerini ve en iyi parametrelerinin
hangilerinin olduğu sonucunu aşağıdaki kod yardımıyla elde ediyoruz.


In [22]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

C:\Users\arife\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


In [23]:
best_parameters

{'criterion': 'mse', 'max_features': 'sqrt', 'n_estimators': 250}

Bu sonuçlar dahilinde, criterion olarak mse, max_features olarak sqrt ve n_estimators olarak da 250
değerlerini kullanmamızın daha başarılı ve doğru sonuçlar verecektir.<br><br>
En son olarak da y_test ve y_predict değerlerimizin kıyaslamasını yapmamız gerekmetedir. Bunun için
regresyon algoritmalarında r kare hesaplaması denilen bir yöntem vardır. Bu yöntem 0 ile 100 arasında
bir değer döndürür ve sonuç 100e yaklaştıkça tahmin edilen değerlerin orjinalleri ile olan yakınlığını
temsil etmketedir. Yani 100e ne kadar yakın olursak o kadar kaliteli sonuçlarımız vardır diyebiliriz.
<br>Bu projemiz içinde aşağıdaki kodu yazarak r kare işlemini devreye sokabiliriz. Çıktı olarak 0 ve 1 arasında
bir sonuç alacağızdır.

In [25]:
from sklearn.metrics import r2_score
rSquare = r2_score(y_test, y_pred)
rSquare

0.9122362500293558

Gördüğümüz üzere aldığımız sonuç 1 değerine oldukça yakın bir yükseklikte. Bu sayede bu modelimizin
iyi bir şekilde öğrendiğini ve gelecek denemelerde yeni veri setleri ile başarılı tahminlerde
bulunabileceğini söyleyebiliriz.